### PV Secondary Target Selection

## sga_semimajor_grid
### Method to create the secondary targeting file for the intermediate points on the semi-major axis of large galaxies (from the SGA) in the BGS sample

##### Author: Kelly Douglass (University of Rochester)

See `/project/projectdirs/desi/target/secondary/README` for output data model

### Target classes
1. End points on the major axis
2. **Points along the major axis**
3. Points off-axis

In [1]:
from astropy.table import Table, Column

import numpy as np

## Parameters

In [2]:
OVERRIDE = True
REF_EPOCH = 2015.5

output_directory = '/project/projectdirs/desi/target/secondary/indata/'

## Target catalogs

[Siena Galaxy Atlas](https://portal.nersc.gov/project/cosmo/temp/ioannis/SGA-html-dr9alpha/SGA-2arcmin.html)

In [3]:
# Target catalog file names

input_directory = '/Users/kellydouglass/Documents/Research/data/SGA/'

input_filename = input_directory + 'SGA_BGS_inspecfootprint.fits'

large_galaxies = Table.read(input_filename, format='fits')

## Set second priority: points along the major axis

Note: Galaxy centers should already be BGS targets

### Within the SGA

We require all galaxies to have a minimum diameter at the 25 magnitude of 20".

In [4]:
SGA_galaxies = large_galaxies[large_galaxies['inSGA']]

### Large galaxies

We target only those galaxies with $R(25) > 1.4'$ (the fiber patrol radius).

In [5]:
max_patrol_radius = 1.4 # arcminutes

lg_SGA = SGA_galaxies[SGA_galaxies['RADIUS_SB25'] >= max_patrol_radius*60]

## Calculate (ra, dec) of major axis end points

We set fiber target locations along the major axis at a distance of $xR_e$, where $x$ varies from 0.5 to 3 in steps of 0.5.

In [6]:
center_ra = lg_SGA['RA']   # degrees
center_dec = lg_SGA['DEC'] # degrees

phi = lg_SGA['pos_angle']*np.pi/180. # degrees -> radians
r50 = lg_SGA['uncor_radius']/3600.   # arcsecs -> degrees

x = np.linspace(0.5,3,6).reshape((1,6))

# Distances along the semi-major axis from the center coordinate for our targets
delta_a = np.dot(r50.reshape((len(r50),1)),x)
delta_a = delta_a.reshape(6,len(delta_a))

# Target positions
fiber1_ra = center_ra + np.dot(delta_a, np.sin(phi).reshape(len(phi),1))
fiber1_dec = center_dec + np.dot(delta_a, np.cos(phi).reshape(len(phi),1))
fiber2_ra = center_ra - np.dot(delta_a, np.sin(phi).reshape(len(phi),1))
fiber2_dec = center_dec - np.dot(delta_a, np.cos(phi).reshape(len(phi),1))

fiber_ra = np.concatenate((np.ravel(fiber1_ra), np.ravel(fiber2_ra)))
fiber_dec = np.concatenate((np.ravel(fiber1_dec), np.ravel(fiber2_dec)))

### Write target list to file

In [7]:
N_targets = len(fiber_ra)

In [8]:
lg_gal_grid = Table([Column(fiber_ra, name='RA'), 
                     Column(fiber_dec, name='DEC'), 
                     Column(np.zeros(N_targets, dtype='>f4'), name='PMRA'), 
                     Column(np.zeros(N_targets, dtype='>f4'), name='PMDEC'), 
                     Column(REF_EPOCH*np.ones(N_targets, dtype='>f4'), name='REF_EPOCH'),
                     Column(OVERRIDE*np.ones(N_targets, dtype='bool'), name='OVERRIDE')])

In [ ]:
lg_gal_grid.write(output_directory + 'sga_semimajor_grid.fits', format='fits')

## Target statistics

In [9]:
print('The number of large galaxies is', len(lg_SGA))
print('The number of targets is', N_targets)

sky_chunk_boolean = np.logical_and.reduce([lg_gal_grid['RA'] > 150, lg_gal_grid['RA'] < 250, 
                                           lg_gal_grid['DEC'] > 0, lg_gal_grid['DEC'] < 50])

num_targets_in_sky_chunk = np.sum(sky_chunk_boolean)

sky_area = 100*50

print('The number of fiber placements in this portion of the sky is', 
      num_targets_in_sky_chunk)
print('The number of fiber placements per square degree is', 
      num_targets_in_sky_chunk/sky_area)

The number of large galaxies is 877
The number of targets is 10524
The number of fiber placements in this portion of the sky is 3642
The number of fiber placements per square degree is 0.7284
